In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import requests
import csv
import time
import seaborn as sns

from config import api_key

In [ ]:
url = f'http://api.sportradar.us/mlb/trial/v6.5/en/games/2018/PST/schedule.json?api_key={api_key}'

resp = requests.get(url).json()

In [ ]:
postseason_games = [x['id'] for x in resp['games']]

In [ ]:
games = list()

i = 0
k = 0

for game in postseason_games:
    try:
        k += 1
        time.sleep(1)
        game_url = f'http://api.sportradar.us/mlb/trial/v6.5/en/games/{game}/pbp.json?api_key={api_key}'
        game_resp = requests.get(game_url).json()
        if 'pitch_x' in game_resp['game']['innings'][1]['halfs'][0]['events'][0]['at_bat']['events'][0]['pitcher'].keys():
            games.append(game)
            i += 1
        print(f'{i}/{k}')
    except Exception as x:
        print(f'{i}/{k}')

In [ ]:
outcome_ids = ['kKL', 'bB', 'aKLAD1', 'aKLAD2', 'aKLAD3', 'aKLAD4', 'oKLT1', 'oKLT2', 'oKLT3', 'oKLT4']
csv_header = ['Umpire', 'Game_ID', 'Home/Away', 'Called Ball/Strike', 'Pitch_X', 'Pitch_Y', 'Pitcher_Hand',
              'Batter_Hand', 'Pitch_Type', 'Pitch_Speed']

with open('Resources/pitch_data.csv', 'w', newline='') as fout:
    csvwriter = csv.writer(fout, delimiter=',')
    csvwriter.writerow(csv_header)
    for game in games:
        time.sleep(1.1)
        game_json = get_game_by_id(game, api_key)
        time.sleep(1.1)
        umpire = get_umpire(game, api_key)
        for inning in game_json['game']['innings'][1:]:
            for half in inning['halfs']:
                t_b = half['half']
                if t_b == 'T':
                    t_b = 'Away'
                else:
                    t_b = 'Home'
                for event in half['events']:
                    try:
                        for pitch in event['at_bat']['events']:
                            if pitch['outcome_id'] in outcome_ids:
                                if pitch['outcome_id'] == 'bB':
                                    call = 'Ball'
                                else:
                                    call = 'Strike'
                                pitch_type = pitch['pitcher']['pitch_type']
                                pitch_speed = pitch['pitcher']['pitch_speed']
                                pitcher_hand = pitch['pitcher']['pitcher_hand']
                                batter_hand = pitch['pitcher']['hitter_hand']
                                pitch_x = pitch['pitcher']['pitch_x']
                                pitch_y = pitch['pitcher']['pitch_y']
                                csvwriter.writerow([umpire, game, t_b, call, pitch_x, pitch_y, pitcher_hand,
                                                    batter_hand, pitch_type, pitch_speed])
                    except:
                        pass
                            
print('done')

In [ ]:
pitch_df = pd.read_csv('Resources/pitch_data.csv')
pitch_df.head()

In [ ]:
pitch_dictionary = {'FA': 'Fastball',
                    'SI': 'Sinker',
                    'CT': 'Cutter',
                    'CU': 'Curveball',
                    'SL': 'Slider',
                    'CH': 'Changeup',
                    'KN': 'Knuckleball',
                    'SP': 'Splitter',
                    'SC': 'Screwball',
                    'FO': 'Forkball',
                    'IB': 'Intentional Ball',
                    'PI': 'Pitchout',}

pitch_df['Theoretical_Call'] = ''
pitch_df['Accurate'] = ''
pitch_df['Strike'] = ''

for i, row in pitch_df.iterrows():
    pitch_x = abs(row['Pitch_X'])
    pitch_y = abs(row['Pitch_Y'])
    pitch_type = row['Pitch Type']
    pitch_df.loc[i, 'Pitch Type'] = pitch_dictionary.get(pitch_type, 'NA')
    if (pitch_x > 100) or (pitch_y > 100):
        pitch_df.loc[i, 'Theoretical_Call'] = 'Ball'
        pitch_df.loc[i, 'Strike'] = False
    else:
        pitch_df.loc[i, 'Theoretical_Call'] = 'Strike'
        pitch_df.loc[i, 'Strike'] = True
    if pitch_df.loc[i, 'Called Ball/Strike'] == pitch_df.loc[i, 'Theoretical_Call']:
        pitch_df.loc[i, 'Accurate'] = True
    else:
        pitch_df.loc[i, 'Accurate'] = False

pitch_df.head()